In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_8bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./model/qwen1.5b-f16",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
    load_in_8bit = load_in_8bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    NVIDIA A30. Num GPUs = 1. Max memory: 23.486 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
alpaca_prompt = """请根据给定问题，按照以下格式生成答案：首先，展示你在思考问题时的过程（用<think>标签包裹），然后给出最终的答案（用<answer>标签包裹）。确保思考过程清晰，逐步推理，并最终给出完整的回答。

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("./Data/law_cot/", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10877 [00:00<?, ? examples/s]

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3409,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/10877 [00:00<?, ? examples/s]

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer_stats = trainer.train()

In [6]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "请归纳这篇文书的大致要点 莫振红与齐爱青、龙书贵借款合同纠纷一审民事判决书 河南省滑县人民法院 民 事 判 决 书 （2018）豫0526民初805号原告莫振红，女，1973年3月3日生，汉族。 委托代理人孟帅、李亚廷（实习），河南卓誉律师事务所律师。 被告齐爱青，女，1968年2月28日生，汉族。 被告龙书贵，男，1964年2月3日生，汉族。 原告莫振红诉被告齐爱青、龙书贵借款合同纠纷一案，本院受理后，依法适用简易程序，公开开庭进行了审理。 原告莫振红的委托代理人孟帅、被告齐爱青到庭参加诉讼，被告龙书贵经传票传唤未到庭参加诉讼。 本案现已审理终结。 原告莫振红诉称，2014年11月11日，二被告向原告借款2万元，并向原告出具了借条一份。 后经多次催要，被告均以各种理由推拖没有偿还原告。 起诉请求依法判令被告偿还原告借款2万元及利息； 诉讼费用由被告承担。 被告齐爱青辩称，借款属实，但中间原告拉走3车砖折抵借款0.8万元，尚欠1.2万元，故被告现在不欠原告2万元钱。 被告龙书贵缺席未答辩。 经审理查明：2014年11月11日，被告齐爱青、龙书贵共同向原告借款2万元，并出具借条一份，内容为：“今借到现金贰万元正￥20000元2014年11月11日齐爱青龙书贵。” 上述借款至今未予偿还。 以上事实，有原告提交的借条及原、被告部分庭审陈述可以证实，上述证据经庭审质证，可以作为认定本案事实的依据。 本院认为，被告齐爱青、龙书贵共同向原告借款2万元，有原告提供的被告齐爱青、龙书贵签名确认的借条予以证实，故被告齐爱青、龙书贵负有共同偿还的责任。 关于利息，因未明确约定，应自原告主张权利时即起诉之日起，按年利率6%计算。 被告齐爱青主张原告拉砖三车即已经折抵借款0.8万元及其他辩解意见，均证据不足，依法不予采信。 根据《中华人民共和国民法通则》第八十四条、第一百零八条，《中华人民共和国民事诉讼法》第六十四条、第一百四十四条的规定，判决如下：被告齐爱青、龙书贵于本判决生效后五日内共同偿还原告莫振红借款人民币2万元及利息（自2018年1月22日起，按照年利率6%计算至本判决限定的履行期限届满之日止）。 如果未按本判决指定的期间履行给付金钱义务，应当依照《中华人民共和国民事诉讼法》第二百五十三条之规定，加倍支付迟延履行期间的债务利息。 案件受理费300元，减半收取150元，被告齐爱青、龙书贵共同负担。 如不服本判决，可在判决送达之日起十五日内，向本院递交上诉状，并按当事人的人数提出副本，上诉于河南省安阳市中级人民法院。 审判员　　张兴政 二〇一八年二月二十二日 书记员　　王舒舒", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1280)

请根据给定问题，按照以下格式生成答案：首先，展示你在思考问题时的过程（用<think>标签包裹），然后给出最终的答案（用<answer>标签包裹）。确保思考过程清晰，逐步推理，并最终给出完整的回答。

### Input:
请归纳这篇文书的大致要点 莫振红与齐爱青、龙书贵借款合同纠纷一审民事判决书 河南省滑县人民法院 民 事 判 决 书 （2018）豫0526民初805号原告莫振红，女，1973年3月3日生，汉族。 委托代理人孟帅、李亚廷（实习），河南卓誉律师事务所律师。 被告齐爱青，女，1968年2月28日生，汉族。 被告龙书贵，男，1964年2月3日生，汉族。 原告莫振红诉被告齐爱青、龙书贵借款合同纠纷一案，本院受理后，依法适用简易程序，公开开庭进行了审理。 原告莫振红的委托代理人孟帅、被告齐爱青到庭参加诉讼，被告龙书贵经传票传唤未到庭参加诉讼。 本案现已审理终结。 原告莫振红诉称，2014年11月11日，二被告向原告借款2万元，并向原告出具了借条一份。 后经多次催要，被告均以各种理由推拖没有偿还原告。 起诉请求依法判令被告偿还原告借款2万元及利息； 诉讼费用由被告承担。 被告齐爱青辩称，借款属实，但中间原告拉走3车砖折抵借款0.8万元，尚欠1.2万元，故被告现在不欠原告2万元钱。 被告龙书贵缺席未答辩。 经审理查明：2014年11月11日，被告齐爱青、龙书贵共同向原告借款2万元，并出具借条一份，内容为：“今借到现金贰万元正￥20000元2014年11月11日齐爱青龙书贵。” 上述借款至今未予偿还。 以上事实，有原告提交的借条及原、被告部分庭审陈述可以证实，上述证据经庭审质证，可以作为认定本案事实的依据。 本院认为，被告齐爱青、龙书贵共同向原告借款2万元，有原告提供的被告齐爱青、龙书贵签名确认的借条予以证实，故被告齐爱青、龙书贵负有共同偿还的责任。 关于利息，因未明确约定，应自原告主张权利时即起诉之日起，按年利率6%计算。 被告齐爱青主张原告拉砖三车即已经折抵借款0.8万元及其他辩解意见，均证据不足，依法不予采信。 根据《中华人民共和国民法通则》第八十四条、第一百零八条，《中华人民共和国民事诉讼法》第六十四条、第一百四十四条的规定，判决如下：被告齐爱青、龙书贵于本判决生效后五日内共同偿还原告莫振红借款人民币2万元及利息（自2018年1月22日起，按照年利率6%计

In [7]:
model.save_pretrained("./model/cot_law_lora_model")  # Local saving
tokenizer.save_pretrained("./model/cot_law_lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('./model/cot_law_lora_model/tokenizer_config.json',
 './model/cot_law_lora_model/special_tokens_map.json',
 './model/cot_law_lora_model/vocab.json',
 './model/cot_law_lora_model/merges.txt',
 './model/cot_law_lora_model/added_tokens.json',
 './model/cot_law_lora_model/tokenizer.json')

In [ ]:
if True: model.save_pretrained_merged("./model/cot_law-f16", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 341.55 out of 503.19 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|███████████████████████████████████████████| 28/28 [00:00<00:00, 30.98it/s]


Unsloth: Saving tokenizer... Done.
Done.
